In [1]:
SCHOOL = "McGill"
data_path = f"bias_processing/data/1/{SCHOOL.lower()}_dataset.csv"
output_path = f"bias_processing/data/3/{SCHOOL.lower()}_dataset_summarizer.csv"

In [ ]:
"""
Load in a csv from Sentiment_Dataset_Maker and add 4x3x3 columns
4 topics ("Israel", "Palestine", "India", "China")
3 hypotheses for sentiment (Positive, Negative, Neutral)
3 levels of granularity
Compute sentiment for entire article
Compute sentiment for each paragraph in an article, take maximum for Positive and Negative, average for neutral
Make 'average' and 'max' options to a function call so we can change if need be
Compute sentiment for each sentence in an article, similar to paragraph approach
Save a new csv with these added columns
"""